# Java Threads and Concurrency Utilities

## Chapter - 6
## Synchronizers

Because it can be difficult to correctly write synchronized code that’s based on synchronized, high-level synchronizers (classes that facilitate common forms of synchronization) are included in the concurrency utilities. In this chapter, I introduce you to the countdown latch, cyclic barrier, exchanger, semaphore, and phaser synchronizers.

## Countdown Latch
The java.util.concurrent.CountDownLatch class implements the countdown latch synchronizer. You initialize a CountDownLatch instance to a specific count by invoking this class’s CountDownLatch(int count) constructor, which throws java.lang. IllegalArgumentException when the value passed to count is negative.

This is a story of a race in which 3 participants Bolt, Honey and Angel participated.

1. All participants came to the starting position
2. Race got started
3. All participants reached to the finishing point

Moral: It does not matter who came first or last, as they all reached to the finishing line. This is multiverse.

Imports and declared the total number of participants.

In [1]:
import java.util.concurrent.CountDownLatch;
import java.util.concurrent.ExecutorService;
import java.util.concurrent.Executors;
final static int NPARTICIPANTS = 3;

Declared 2 countdown latch.
1. startSignal will be used to start all NPARTICIPANTS at the same time. Like the race starts here, this is the starting point
2. doneSignal will be used to finish the race, like all the participants reached the finishing line and race got over.

In [2]:
final CountDownLatch startSignal = new CountDownLatch(1);
final CountDownLatch doneSignal = new CountDownLatch(NPARTICIPANTS);

This method is used to report the positions of the participants.

In [3]:
static void report(String s) {
        System.out.println(System.currentTimeMillis() + ": " + Thread.currentThread() + ": " + s);
    }

In [4]:
Runnable r = () -> {
    try {
        report("Participant entered the race <run() method> and is ready at starting point");
        startSignal.await(); // wait until told to
        report("Running in the race. <Started doing some work>"); // Proceed
        doneSignal.countDown(); // reduce count on which main thread is waiting
    } catch (InterruptedException ie) {
    }
};

In [5]:
ExecutorService executor = Executors.newFixedThreadPool(NPARTICIPANTS);

for (int i = 0; i < NPARTICIPANTS; i++)
    executor.execute(r);

try {
    System.out.println("Come on all the participants, the race is going to be started in 1 second, get ready");
    Thread.sleep(1000); // sleep for 1 second
    startSignal.countDown(); // let all participants proceed
    System.out.println("Race started. 3 2 1 GO... Cheers...");
    doneSignal.await(); // wait for all participants to finish
    System.out.println("All the participants reached the finishing line, and all won the race. Cheers... Race is over.");
    executor.shutdownNow();
} catch (InterruptedException ie) {
    System.err.println(ie);
}

1594577757660: Thread[pool-1-thread-1,5,main]: Participant entered the race <run() method> and is ready at starting point
1594577757661: Thread[pool-1-thread-3,5,main]: Participant entered the race <run() method> and is ready at starting point
1594577757661: Thread[pool-1-thread-2,5,main]: Participant entered the race <run() method> and is ready at starting point
Come on all the participants, the race is going to be started in 1 second, get ready
Race started. 3 2 1 GO... Cheers...
1594577758771: Thread[pool-1-thread-1,5,main]: Running in the race. <Started doing some work>
1594577758772: Thread[pool-1-thread-2,5,main]: Running in the race. <Started doing some work>
1594577758771: Thread[pool-1-thread-3,5,main]: Running in the race. <Started doing some work>
All the participants reached the finishing line, and all won the race. Cheers... Race is over.


## Custom Count Down Latch
Implement a working count down latch for POC

In [6]:
class CustomCountDownLatch {
    int counter;

    public CustomCountDownLatch(int counter) {
        this.counter = counter;
    }

    public synchronized void await() throws InterruptedException {
        if (counter > 0) {
            wait();
        }
    }

    /**
     * method will decrement the counter by 1 each time
     */
    public synchronized void countDown() {
        counter--;
        if (counter == 0) {
            notifyAll();
        }
    }
}

In [7]:
CustomCountDownLatch countDownLatch = new CustomCountDownLatch(3);

In [8]:
Runnable runnableCustomCDL = () -> {
    System.out.println("Working : " + Thread.currentThread().getName());
    countDownLatch.countDown();
    try {
        Thread.sleep(2000);
    } catch (InterruptedException e) {
        e.printStackTrace();
    }
};

In [9]:
Thread t1 = new Thread(runnableCustomCDL, "T1");
Thread t2 = new Thread(runnableCustomCDL, "T2");
Thread t3 = new Thread(runnableCustomCDL, "T3");

In [10]:
t1.start();
t2.start();
t3.start();

try {
    System.out.println("Waiting for all the threads to finish");
    countDownLatch.await();
} catch (InterruptedException e) {
    e.printStackTrace();
}

System.out.println("Finished work load of all the threads.");

Working : T1
Working : T2
Working : T3
Waiting for all the threads to finish
Finished work load of all the threads.


## Cyclic Barriers

Cyclic barriers are useful in parallel decomposition scenarios, where a lengthy task is divided into subtasks whose individual results are later merged into the overall result of the task. 

In [11]:
import java.util.ArrayList;
import java.util.Collections;
import java.util.List;
import java.util.Random;
import java.util.concurrent.BrokenBarrierException;
import java.util.concurrent.CyclicBarrier;

In [12]:
class Solver {
    final int N;
    final float[][] data;
    final CyclicBarrier barrier;

    class Worker implements Runnable {
        int myRow;
        boolean done = false;

        Worker(int row) {
            myRow = row;
        }

        boolean done() {
            return done;
        }

        void processRow(int myRow) {
            System.out.println("Processing row: " + myRow);
            for (int i = 0; i < N; i++)
                data[myRow][i] *= 10;
            done = true;
        }

        @Override
        public void run() {
            while (!done()) {
                processRow(myRow);

                try {
                    barrier.await();
                } catch (InterruptedException ie) {
                    return;
                } catch (BrokenBarrierException bbe) {
                    return;
                }
            }
        }
    }

    public Solver(float[][] matrix) {
        data = matrix;
        N = matrix.length;
        barrier = new CyclicBarrier(N, new Runnable() {
            @Override
            public void run() {
                mergeRows();
            }
        });
        for (int i = 0; i < N; ++i)
            new Thread(new Worker(i)).start();

        waitUntilDone();
    }

    void mergeRows() {
        System.out.println("merging");
        synchronized ("abc") {
            "abc".notify();
        }
    }

    void waitUntilDone() {
        synchronized ("abc") {
            try {
                System.out.println("main thread waiting");
                "abc".wait();
                System.out.println("main thread notified");
            } catch (InterruptedException ie) {
                System.out.println("main thread interrupted");
            }
        }
    }
}

In [13]:
static void dump(float[][] matrix) {
    for (int row = 0; row < matrix.length; row++) {
        for (int col = 0; col < matrix[0].length; col++)
            System.out.print(matrix[row][col] + " ");
        System.out.println();
    }
}

In [14]:
float[][] matrix = new float[3][3];
int counter = 0;
for (int row = 0; row < matrix.length; row++)
    for (int col = 0; col < matrix[0].length; col++)
        matrix[row][col] = counter++;
dump(matrix);
System.out.println();
Solver solver = new Solver(matrix);
System.out.println();
dump(matrix);

0.0 1.0 2.0 
3.0 4.0 5.0 
6.0 7.0 8.0 

Processing row: 0
main thread waiting
Processing row: 2
Processing row: 1
merging
main thread notified

0.0 10.0 20.0 
30.0 40.0 50.0 
60.0 70.0 80.0 


## Exchangers
An exchanger provides a synchronization point where threads can swap objects. Each thread presents some object on entry to the exchanger’s exchange() method, matches with a partner thread, and receives its partner’s object on return. Exchangers can be useful in applications such as genetic algorithms (see http://en.wikipedia.org/wiki/Genetic_algorithm) and pipeline designs.

In [15]:
// Look for some code example.

## Semaphores
A semaphore maintains a set of permits for restricting the number of threads that can access a limited resource. A thread attempting to acquire a permit when no permits are available blocks until some other thread releases a permit.

■ Note Semaphores whose current values can be incremented past 1 are known as counting semaphores, whereas semaphores whose current values can be only 0 or 1 are known as binary semaphores or mutexes. in either case, the current value cannot be negative.


Look for some good code example.

In [16]:
// max 4 people
static Semaphore semaphore = new Semaphore(4);

In [17]:
class MyATMThread extends Thread {

    String name = "";

    MyATMThread(String name) {
        this.name = name;
    }

    public void run() {

        try {

            System.out.println(name + " : acquiring lock...");
            System.out.println(name + " : available Semaphore permits now: " + semaphore.availablePermits());

            semaphore.acquire();
            System.out.println(name + " : got the permit!");

            try {

                for (int i = 1; i <= 5; i++) {

                    System.out.println(name + " : is performing operation " + i + ", available Semaphore permits : "
                            + semaphore.availablePermits());

                    // sleep 1 second
                    Thread.sleep(1000);

                }

            } finally {

                // calling release() after a successful acquire()
                System.out.println(name + " : releasing lock...");
                semaphore.release();
                System.out.println(name + " : available Semaphore permits now: " + semaphore.availablePermits());

            }

        } catch (InterruptedException e) {

            e.printStackTrace();

        }

    }

}

In [18]:
System.out.println("Total available Semaphore permits : "
        + semaphore.availablePermits());

MyATMThread t1 = new MyATMThread("A");
t1.start();

MyATMThread t2 = new MyATMThread("B");
t2.start();

MyATMThread t3 = new MyATMThread("C");
t3.start();

MyATMThread t4 = new MyATMThread("D");
t4.start();

MyATMThread t5 = new MyATMThread("E");
t5.start();

MyATMThread t6 = new MyATMThread("F");
t6.start();

Total available Semaphore permits : 4
A : acquiring lock...
A : available Semaphore permits now: 4
A : got the permit!
A : is performing operation 1, available Semaphore permits : 3
B : acquiring lock...
B : available Semaphore permits now: 3
B : got the permit!
B : is performing operation 1, available Semaphore permits : 2
C : acquiring lock...
C : available Semaphore permits now: 2
C : got the permit!
C : is performing operation 1, available Semaphore permits : 1
D : acquiring lock...
D : available Semaphore permits now: 1
D : got the permit!
D : is performing operation 1, available Semaphore permits : 0
E : acquiring lock...
E : available Semaphore permits now: 0
F : acquiring lock...
F : available Semaphore permits now: 0


## Phasers
A phaser is a more flexible cyclic barrier. Like a cyclic barrier, a phaser lets a group of threads wait on a barrier; these threads continue after the last thread arrives. A phaser
also offers the equivalent of a barrier action. Unlike a cyclic barrier, which coordinates a fixed number of threads, a phaser can coordinate a variable number of threads, which can register at any time. To implement this capability, a phaser uses phases and phase numbers.

A phase is the phaser’s current state, and this state is identified by an integer-based phase number. When the last of the registered threads arrives at the phaser barrier, a phaser advances to the next phase and increments its phase number by 1.

In [19]:
static void runTasks(List<Runnable> tasks) {
    final Phaser phaser = new Phaser(1); // "1" (register self)
    // create and start threads
    for (final Runnable task : tasks) {
        phaser.register();
        Runnable r = () -> {
//             try {
//                 Thread.sleep(500);
//             } catch (InterruptedException ie) {
//                 System.out.println("interrupted thread");
//             }
            phaser.arriveAndAwaitAdvance(); // await the ...
                                            // creation of ...
                                            // all tasks
            task.run();
        };
        Executors.newSingleThreadExecutor().execute(r);
    }
    // allow threads to start and deregister self
    phaser.arriveAndDeregister();
}

In [20]:
List<Runnable> tasks = new ArrayList<>();
tasks.add(() -> System.out.printf("%s running at %d%n", Thread.currentThread().getName(),
        System.currentTimeMillis()));
tasks.add(() -> System.out.printf("%s running at %d%n", Thread.currentThread().getName(),
        System.currentTimeMillis()));
runTasks(tasks);

A : is performing operation 2, available Semaphore permits : 0
B : is performing operation 2, available Semaphore permits : 0
pool-3-thread-1 running at 1594577763108
pool-2-thread-1 running at 1594577763109


## Summary
Java provides the synchronized keyword for synchronizing thread access to critical sections. Because it can be difficult to correctly write synchronized code that’s based on synchronized, high-level synchronizers are included in the concurrency utilities.

A __countdown latch__ causes one or more threads to wait at a “gate” until another thread opens this gate, at which point these other threads can continue. It consists of a count and operations for “causing a thread to wait until the count reaches zero” and “decrementing the count.”

A __cyclic barrier__ lets a set of threads wait for each other to reach a common barrier point. The barrier is cyclic because it can be reused after the waiting threads are released. This synchronizer is useful in applications involving a fixed-size party of threads that must occasionally wait for each other.

An __exchanger__ provides a synchronization point where threads can swap objects. Each thread presents some object on entry to the exchanger’s exchange() method, matches with a partner thread, and receives its partner’s object on return.

A __semaphore__ maintains a set of permits for restricting the number of threads that can access a limited resource. A thread attempting to acquire a permit when no permits are available blocks until some other thread releases a permit.

A __phaser__ is a more flexible cyclic barrier. Like a cyclic barrier, a phaser lets a group of threads wait on a barrier; these threads continue after the last thread arrives. A phaser also offers the equivalent of a barrier action. Unlike a cyclic barrier, which coordinates a fixed number of threads, a phaser can coordinate a variable number of threads, which can register at any time. To implement this capability, a phaser uses phases and phase numbers.

Chapter 7 presents the Locking Framework.